In [1]:
import re
import os
from time import sleep

import numpy as np
import pandas as pd
import time
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup as bs

In [2]:
# создание директории для csv файлов
os.makedirs('data/', exist_ok=True)

In [3]:
# def parse_page(page=0, max_articles_per_page=15):
    '''
        Функция для парсинга названий и ссылок статей с указанной страницы.
        Возвращает все созданные статьи в виде списка со словарями.
    '''
    url = f'https://nauchforum.ru/archive/article/all?page={page}'
    base_url = 'https://nauchforum.ru'
    # Получение сырых данных по ссылке и преобразование в формат BeautifulSoup
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')

    data = []
    # Сбор всех объектов статей по названию класса
    articles = soup.find('ul', class_='archive-list').findAll('li')

    for i, article in enumerate(articles[:max_articles_per_page]):
        try:
            # Сбор заголовка
            title = article.find('a').text
            # Сбор авторов
            authors = article.findAll('div')[1].text.replace('Авторы: ', '').strip().split()

            # Если у статьи указан научный руководитель, добавить и очистить от лишнего
            if len(article.findAll('div')) > 2:
                academic_advisor = article.findAll('div')[2].text.replace('Научный руководитель: ', '').strip()
            else:
                academic_advisor = None

            # Сбор ссылки на статью
            link = article.find('a')['href'] if article.find('a')['href'] else None
            link = base_url + link
            data.append({'title': title, 'authors': authors, 'academic_advisor': academic_advisor, 'link': link})
        # Обработка ошибок в случае неудачи
        except Exception as e:
            print(f"Не удалось пропарсить статью {i + 1}. Ошибка: {e}")
            print(article)

    return data

In [4]:
arts = parse_page()

In [5]:
arts[:2]

[{'title': 'ЖЕНСКАЯ ПРЕСТУПНОСТЬ В РОССИЙСКОЙ ФЕДЕРАЦИИ',
  'authors': ['Кечаева',
   'Юлия',
   'Владимировна,',
   'Кротова',
   'Виктория',
   'Алексеевна,',
   'Калаева',
   'Ангелина',
   'Вячеславовна,',
   'Ларина',
   'Арина',
   'Андреевна'],
  'academic_advisor': 'Авдеева Анна Константиновна',
  'link': 'https://nauchforum.ru/studconf/science/275/155428'},
 {'title': 'БИОЛОГИЧЕСКИЕ НАУКИ',
  'authors': ['Харитонов', 'Фёдор', 'Иванович'],
  'academic_advisor': None,
  'link': 'https://nauchforum.ru/studconf/science/275/155413'}]

In [6]:
arts[14]

{'title': 'ПОНЯТИЕ И НЕКОТОРЫЕ ОСОБЕННОСТИ ДОГОВОРА АРЕНДЫ НЕДВИЖИМОСТИ',
 'authors': ['Уздемир', 'Валерия', 'Святославовна'],
 'academic_advisor': None,
 'link': 'https://nauchforum.ru/studconf/science/274/155001'}

Количество собранных статей:

In [7]:
len(arts)

15

Проверка, что данные находятся правильно:

In [8]:
response = requests.get('https://nauchforum.ru/studconf/science/275/155428')
soup = bs(response.content, 'html.parser')

In [9]:
soup.find('div', class_='article_head clearfix').find('a').text.strip()

'CCLXXV Студенческая международная научно-практическая конференция «Молодежный научный форум»'

In [10]:
soup.find('div', class_='article_head clearfix').findAll('p')[1].text.replace('Секция: ', '').strip()

'Юриспруденция'

Сборник:

In [11]:
soup.find('div', class_='article_head clearfix').find('div', class_='action').findAll('a')[1]['href']

'/archive/MNF_interdisciplinarity/43%28275%29.pdf'

Количество голосов:

In [12]:
soup.find('div', class_='article_head clearfix').find('div', class_='count').find('span').text

'0'

In [13]:
soup.find('div', class_='body-content').text[:500]

'\n\xa0\nFEMALE CRIMINALITY IN CRIMINAL LAW\n\xa0\nYulia Kechaeva\nStudent, GBPOU Nizhny Novgorod Provincial College, Russia, Nizhny Novgorod\nArina Larina \nStudent, GBPOU Nizhny Novgorod Provincial College, Russia, Nizhny Novgorod\nVictoria Krotova \nStudent, GBPOU Nizhny Novgorod Provincial College, Russia, Nizhny Novgorod\nAngelina Kalaeva \nStudent, GBPOU Nizhny Novgorod Provincial College, Russia, Nizhny Novgorod\nAnna Avdeeva \nScientific supervisor, Teacher, GBPOU Nizhny Novgorod Provincial College, Russia,'

In [4]:
def parse_article(title, authors, academic_advisor, url):
    '''
        Функция для парсинга статьи.
        Берёт имеющиеся данные: заголовок, авторов, научного руководителя и ссылку.
        По ссылке собирает данные и возвращает словарь.
    '''
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')

    # Инициализация значений на случай, если их не удастся собрать
    conference = section = collection_url = n_votes = text = None

    try:
        # Сбор зоголовочной части статьи
        article_header = soup.find('div', class_='article_head clearfix')
        if article_header:
            if article_header.find('a'):
                # Сбор названия конференции
                conference = soup.find('div', class_='article_head clearfix').find('a').text.strip()
            if article_header.findAll('p'):
                # Сбор названия секции
                section = article_header.findAll('p')[1].text.replace('Секция: ', '').strip()

            # Полчение данных с кнопок внутри зазоловочной части
            article_header_action = article_header.find('div', class_='action')
            if article_header_action and article_header_action.find('a', class_='btn'):
                # Получение ссылки на сборник, где была размещена статья
                collection_url = article_header_action.find('a', class_='btn')['href']

            # Сбор количества голосов за статью
            article_header_cnt = article_header.find('div', class_='count')
            if article_header_cnt and article_header_cnt.find('span'):
                n_votes = article_header_cnt.find('span').text

        # Сбор текста статьи без обработки
        if soup.find('div', class_='body-content'):
            text = soup.find('div', class_='body-content').text
    # Обработка ошибок в случае неудачи
    except Exception as e:
        print(f"Не удалось получить часть данных для статьи {title} с url: {url}. Ошибка: {e}")

    return {
        'title': title,
        'authors': authors,
        'academic_advisor': academic_advisor,
        'url': url,
        'conference': conference,
        'section': section,
        'collection_url': collection_url,
        'n_votes': n_votes,
        'text': text,
    }

In [16]:
arts[0]

{'title': 'ЖЕНСКАЯ ПРЕСТУПНОСТЬ В РОССИЙСКОЙ ФЕДЕРАЦИИ',
 'authors': ['Кечаева',
  'Юлия',
  'Владимировна,',
  'Кротова',
  'Виктория',
  'Алексеевна,',
  'Калаева',
  'Ангелина',
  'Вячеславовна,',
  'Ларина',
  'Арина',
  'Андреевна'],
 'academic_advisor': 'Авдеева Анна Константиновна',
 'link': 'https://nauchforum.ru/studconf/science/275/155428'}

In [17]:
test_article = tuple(arts[0].values())
test_article

('ЖЕНСКАЯ ПРЕСТУПНОСТЬ В РОССИЙСКОЙ ФЕДЕРАЦИИ',
 ['Кечаева',
  'Юлия',
  'Владимировна,',
  'Кротова',
  'Виктория',
  'Алексеевна,',
  'Калаева',
  'Ангелина',
  'Вячеславовна,',
  'Ларина',
  'Арина',
  'Андреевна'],
 'Авдеева Анна Константиновна',
 'https://nauchforum.ru/studconf/science/275/155428')

In [18]:
parse_article(*test_article)

{'title': 'ЖЕНСКАЯ ПРЕСТУПНОСТЬ В РОССИЙСКОЙ ФЕДЕРАЦИИ',
 'authors': ['Кечаева',
  'Юлия',
  'Владимировна,',
  'Кротова',
  'Виктория',
  'Алексеевна,',
  'Калаева',
  'Ангелина',
  'Вячеславовна,',
  'Ларина',
  'Арина',
  'Андреевна'],
 'academic_advisor': 'Авдеева Анна Константиновна',
 'url': 'https://nauchforum.ru/studconf/science/275/155428',
 'conference': 'CCLXXV Студенческая международная научно-практическая конференция «Молодежный научный форум»',
 'section': 'Юриспруденция',
 'collection_url': '/studconf/science',
 'n_votes': '0',
 'text': "\n\xa0\nFEMALE CRIMINALITY IN CRIMINAL LAW\n\xa0\nYulia Kechaeva\nStudent, GBPOU Nizhny Novgorod Provincial College, Russia, Nizhny Novgorod\nArina Larina \nStudent, GBPOU Nizhny Novgorod Provincial College, Russia, Nizhny Novgorod\nVictoria Krotova \nStudent, GBPOU Nizhny Novgorod Provincial College, Russia, Nizhny Novgorod\nAngelina Kalaeva \nStudent, GBPOU Nizhny Novgorod Provincial College, Russia, Nizhny Novgorod\nAnna Avdeeva \nSci

___

In [5]:
def save_to_csv(df, page_num):
    '''
        Функция для сохранения Pandas DataFrame в csv файл.
    '''
    output_file = os.path.join('data/', f'data_page_{page_num}.csv')
    df.to_csv(output_file, index=False)

In [6]:
def parse(num_articles, start_page=0):
    '''
    Основная функция для парсинга.
    Принимает нужное количество статей и начальную страницу (счёт начинается с 0).
    '''
    max_pages = 778
    all_articles = []
    article_count = 0
    # Флаг для остановки парсинга
    stop_parsing = False

    # Парсинг каждой страницы
    for page_num in range(start_page, max_pages + 1):
        # Выход из цикла, если активирован флаг остановки
        if stop_parsing:
            break

        print(f"Страница: {page_num}")
        articles = parse_page(page_num)

        # Инициализация списка со статьями в Pandas DataFrame и их парсинг 
        article_data = []
        for article in articles:
            # Активация флага остановки парсинга при достижении нужного количества статей
            if article_count >= num_articles:
                stop_parsing = True
                break

            art_title = article['title']
            print(f"Парсинг статьи {article_count + 1}: {art_title}")

            # Парсинг текущей статьи и обработка ошибки
            try:
                article_info = parse_article(*article.values())
            except Exception as e:
                print(f'Соединение с {article["link"]} провалилась. Ошибка: {e}')
                time.sleep(2)

            article_data.append(article_info)
            article_count += 1
            # Задержка, чтобы не перегружать сайт
            sleep(0.005)

        if articles:
            df = pd.DataFrame(article_data)
            save_to_csv(df, page_num)
            all_articles.extend(article_data)
        else:
            print(f"Статей на странице {page_num} нет")
            break

    print("Парсинг завершён")

Провека функции:

In [21]:
parse(100)

Страница: 0
Парсинг статьи 1: ЖЕНСКАЯ ПРЕСТУПНОСТЬ В РОССИЙСКОЙ ФЕДЕРАЦИИ
Парсинг статьи 2: БИОЛОГИЧЕСКИЕ НАУКИ
Парсинг статьи 3: ПЕРЕДАЧА ОПРЕДЕЛЕННЫХ ПОЛНОМОЧИЙ В СФЕРЕ ОБРАЗОВАНИЯ НА УРОВЕНЬ СУБЪЕКТОВ РОССИИ
Парсинг статьи 4: ВЛИЯНИЕ ПАНДЕМИИ КОРОНАВИРУСА COVID-19 НА РАСПРОСТРАНЁННОСТЬ ЗАБОЛЕВАЕМОСТИ ОРГАНОВ ДЫХАНИЯ У ДЕТЕЙ ГОРОДА НИЖНЕВАРТОВСКА ЗА 2018-2020 ГОДА
Парсинг статьи 5: ПРЕКРАЩЕНИЕ УГОЛОВНОГО ДЕЛА В СВЯЗИ С ПРИМИРЕНИЕМ СТОРОН ПО ДЕЛАМ ЧАСТНОГО ОБВИНЕНИЯ
Парсинг статьи 6: ТЕНДЕНЦИИ ПРИМЕНЕНИЯ ЦИФРОВЫХ ТЕХНОЛОГИЙ В ПРАВООХРАНИТЕЛЬНОЙ ДЕЯТЕЛЬНОСТИ
Парсинг статьи 7: ПРЕСТУПЛЕНИЯ ПРОТИВ ЖИЗНИ ПО УГОЛОВНОМУ ПРАВУ РОССИИ
Парсинг статьи 8: ЭФФЕКТИВНОСТЬ ВНЕДРЕНИЯ АВТОМАТИЗИРОВАННОЙ ИН-ФОРМАЦИОННО-ИЗМЕРИТЕЛЬНОЙ СИСТЕМЫ КОММЕРЧЕСКО-ГО УЧЕТА
Парсинг статьи 9: ПОВЫШЕНИЕ НАДЕЖНОСТИ СИСТЕМ ЭЛЕКТРОСНАБЖЕНИЯ ПОТРЕБИТЕЛЕЙ
Парсинг статьи 10: ИСТОРИЯ ДЕЯТЕЛЬНОСТИ ЧЕРНОМОРСКОГО ЯХТ-КЛУБА: ПРОВЕДЕНИЕ ПАРУСНЫХ ГОНОК ЗА 1897 ГОД
Парсинг статьи 11: ЭКОНОМИКА НЕРАВЕНСТВА: ГЛУБИННЫЙ АНАЛИЗ ПРИЧИН,

KeyboardInterrupt: 

Проверка правильной обработки последней страницы с неполным списком статей:

In [22]:
parse_page(778)

[{'title': 'ИДЕНТИЧНОСТЬ СОВРЕМЕННОЙ РОССИИ',
  'authors': ['Левина',
   'Виктория,',
   'Аппазова',
   'Зейнеб',
   ',',
   'Тучина',
   'Оксана',
   'Роальдовна'],
  'academic_advisor': None,
  'link': 'https://nauchforum.ru/studconf/gum/i/154'},
 {'title': 'ОСОБЕННОСТИ СОЦИАЛЬНОЙ АДАПТАЦИИ ДЕТЕЙ-ИНВАЛИДОВ С СИНДРОМОМ ШЕРЕШЕВСКОГО-ТЕРНЕРА',
  'authors': ['Матвеева',
   'Ольга',
   'Витальевна,',
   'Кулиниченко',
   'Виктория',
   'Алексеевна'],
  'academic_advisor': None,
  'link': 'https://nauchforum.ru/studconf/gum/i/153'},
 {'title': 'ОСОБЕННОСТИ ЛИЧНОСТНОЙ ИДЕНТИЧНОСТИ У ПОДРОСТКОВ (НА ПРИМЕРЕ КОРРЕКЦИОННОЙ И ОБЩЕОБРАЗОВАТЕЛЬНОЙ ШКОЛЫ)',
  'authors': ['Кукуев',
   'Евгений',
   'Анатольевич,',
   'Сахарова',
   'Юлия',
   'Павловна'],
  'academic_advisor': None,
  'link': 'https://nauchforum.ru/studconf/gum/i/152'},
 {'title': 'АНАЛИЗ ЭТНИЧЕСКИХ КОНСТАНТОВ КАК МЕТОД КОНЦЕПТУАЛЬНЫХ ИССЛЕДОВАНИЙ',
  'authors': ['Капенова',
   'Жанарсын',
   'Жуматовна,',
   'Такенова',
   'Гульнур

In [23]:
parse(1000, 778)

Страница: 778
Парсинг статьи 1: ИДЕНТИЧНОСТЬ СОВРЕМЕННОЙ РОССИИ
Парсинг статьи 2: ОСОБЕННОСТИ СОЦИАЛЬНОЙ АДАПТАЦИИ ДЕТЕЙ-ИНВАЛИДОВ С СИНДРОМОМ ШЕРЕШЕВСКОГО-ТЕРНЕРА
Парсинг статьи 3: ОСОБЕННОСТИ ЛИЧНОСТНОЙ ИДЕНТИЧНОСТИ У ПОДРОСТКОВ (НА ПРИМЕРЕ КОРРЕКЦИОННОЙ И ОБЩЕОБРАЗОВАТЕЛЬНОЙ ШКОЛЫ)
Парсинг статьи 4: АНАЛИЗ ЭТНИЧЕСКИХ КОНСТАНТОВ КАК МЕТОД КОНЦЕПТУАЛЬНЫХ ИССЛЕДОВАНИЙ
Парсинг статьи 5: СПОСОБЫ ДОСТИЖЕНИЯ ЭКВИВАЛЕНТНОСТИ ПЕРЕВОДА КАТАЛОГОВ ЖЕНСКОЙ ОДЕЖДЫ (НА МАТЕРИАЛЕ АНГЛИЙСКОГО И РУССКОГО ЯЗЫКОВ)
Парсинг статьи 6: ЧЕЛОВЕК И КИБЕРПРОСТАНСТВО
Парсинг завершён


In [24]:
parse(1000, 779)

Парсинг завершён


___
Финальный парсинг:

In [ ]:
parse(50000)

Страница: 0
Парсинг статьи 1: ЖЕНСКАЯ ПРЕСТУПНОСТЬ В РОССИЙСКОЙ ФЕДЕРАЦИИ
Парсинг статьи 2: БИОЛОГИЧЕСКИЕ НАУКИ
Парсинг статьи 3: ПЕРЕДАЧА ОПРЕДЕЛЕННЫХ ПОЛНОМОЧИЙ В СФЕРЕ ОБРАЗОВАНИЯ НА УРОВЕНЬ СУБЪЕКТОВ РОССИИ
Парсинг статьи 4: ВЛИЯНИЕ ПАНДЕМИИ КОРОНАВИРУСА COVID-19 НА РАСПРОСТРАНЁННОСТЬ ЗАБОЛЕВАЕМОСТИ ОРГАНОВ ДЫХАНИЯ У ДЕТЕЙ ГОРОДА НИЖНЕВАРТОВСКА ЗА 2018-2020 ГОДА
Парсинг статьи 5: ПРЕКРАЩЕНИЕ УГОЛОВНОГО ДЕЛА В СВЯЗИ С ПРИМИРЕНИЕМ СТОРОН ПО ДЕЛАМ ЧАСТНОГО ОБВИНЕНИЯ
Парсинг статьи 6: ТЕНДЕНЦИИ ПРИМЕНЕНИЯ ЦИФРОВЫХ ТЕХНОЛОГИЙ В ПРАВООХРАНИТЕЛЬНОЙ ДЕЯТЕЛЬНОСТИ
Парсинг статьи 7: ПРЕСТУПЛЕНИЯ ПРОТИВ ЖИЗНИ ПО УГОЛОВНОМУ ПРАВУ РОССИИ
Парсинг статьи 8: ЭФФЕКТИВНОСТЬ ВНЕДРЕНИЯ АВТОМАТИЗИРОВАННОЙ ИН-ФОРМАЦИОННО-ИЗМЕРИТЕЛЬНОЙ СИСТЕМЫ КОММЕРЧЕСКО-ГО УЧЕТА
Парсинг статьи 9: ПОВЫШЕНИЕ НАДЕЖНОСТИ СИСТЕМ ЭЛЕКТРОСНАБЖЕНИЯ ПОТРЕБИТЕЛЕЙ
Парсинг статьи 10: ИСТОРИЯ ДЕЯТЕЛЬНОСТИ ЧЕРНОМОРСКОГО ЯХТ-КЛУБА: ПРОВЕДЕНИЕ ПАРУСНЫХ ГОНОК ЗА 1897 ГОД
Парсинг статьи 11: ЭКОНОМИКА НЕРАВЕНСТВА: ГЛУБИННЫЙ АНАЛИЗ ПРИЧИН,

In [7]:
parse(50000, 665)

Страница: 665
Парсинг статьи 1: ОРГАНИЗАЦИОННАЯ КУЛЬТУРА И ЕЕ СОВЕРШЕНСТВОВАНИЕ
Парсинг статьи 2: ЭКЗИСТЕНЦИАЛЬНЫЙ МОТИВ В ОБРАЗЕ ГЛАВНОГО ГЕРОЯ (ПО РОМАНАМ Ф. КАФКИ «ЗАМОК» И К. ВАГИНОВА «КОЗЛИНАЯ ПЕСНЬ»
Парсинг статьи 3: ВОСПИТАТЕЛЬНАЯ РАБОТА С МЛАДШИМИ ШКОЛЬНИКАМИ КАК СРЕДСТВО ФОРМИРОВАНИЯ У НИХ ПОЗНАВАТЕЛЬНОГО ИНТЕРЕСА
Парсинг статьи 4: ОСОБЕННОСТИ И ЗАДАЧИ ПАТРИОТИЧЕСКОГО ВОСПИТАНИЯ МЛАДШИХ ШКОЛЬНИКОВ
Парсинг статьи 5: ПРАВОВОЕ РЕГУЛИРОВАНИЕ ПРОЦЕССА НАЗНАЧЕНИЯ СОЦИАЛЬНОЙ СТИПЕНДИИ
Парсинг статьи 6: К ВОПРОСУ О СУЩНОСТИ УГОЛОВНО-ПРОЦЕССУАЛЬНОГО ДОКАЗЫВАНИЯ
Парсинг статьи 7: ЯЗЫКОВАЯ ЛИЧНОСТЬ Н. ГУМИЛЕВА КАК ПЕРЕВОДЧИКА НА МАТЕРИАЛЕ ПЕРЕВОДА СОНЕТА У. ШЕКСПИРА “SONNET 17”
Парсинг статьи 8: ОБРАЗ МАЛЬЧИШКИ-ПОДРОСТКА В ПОВЕСТИ А.П. ГАЙДАРА «ТИМУР И ЕГО КОМАНДА»: ТИМУР И КВАКИН
Парсинг статьи 9: ВЛИЯНИЕ ФИЗИЧЕСКОЙ КУЛЬТУРЫ НА СОЦИАЛЬНУЮ СТОРОНУ ЖИЗНИ
Парсинг статьи 10: ИСТОРИКО-ЛОГИЧЕСКИЙ АНАЛИЗ ФАКТОРОВ, УЧИТЫВАЕМЫХ ПРИ СТРОИТЕЛЬСТВЕ СОВРЕМЕННЫХ СТАДИОНОВ
Парсинг статьи 11: РЕАЛИЗАЦИ

Объединение всех промежуточных файлов .csv в один:

In [8]:
data_folder = 'data/'
csv_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('.csv')]
df_list = [pd.read_csv(file) for file in csv_files]
all_df = pd.concat(df_list, ignore_index=True)
all_df.to_csv(os.path.join(data_folder, 'full_parsed_data.csv'), index=False)